In [2]:
from torch import nn
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from collections import defaultdict, Counter
from torch.utils.data import DataLoader
import time

# Building your own Tokenizer

In [31]:
tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split="train")
num_class = len(set([label for (label, text) in train_iter]))

ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}



In [39]:
device = 'cpu'
num_samples = -1
# normalize and tokenize 
def yield_tokens(data_iter):
    for _, text in data_iter:
        text = text.replace("\\"," ").replace("(","").replace(")","").replace(",","").replace("-"," ").lower()
        yield tokenizer(text)
        

token_iter = yield_tokens(train_iter)
token_count = defaultdict(int)
while True: 
    try:
        tokens = next(token_iter)
        num_samples += 1
        for x in tokens:
            token_count[x] += 1         
    except StopIteration as e:
        break

In [41]:
print("number of samples %d" %num_samples)

number of samples 119999


# Tokenizer

In [ ]:
threshold = 20 # min number of counts for a word to be a token
token_map = {}
tid = 0
token_map['<unk>'] = tid
for k,v in token_count.items():
    if v >= threshold : 
        tid += 1
        token_map[k] = tid
        
text_pipeline  = lambda x: [token_map[y] if y in token_map else 0 for y in tokenizer(x)  ]
label_pipeline = lambda x: int(x) - 1
vocab_size = len(token_map)+1

text = 'my tokenizer is nice'
text_pipeline(text)

In [ ]:
def collate_batch(batch): # collate_fn will take care of padding
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64) 
        
        label_list.append(label_pipeline(_label))# adding 1 to label 
        text_list.append(processed_text) # tokenized texts
        offsets.append(processed_text.size(0)) # text size for padding 
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets    = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list  = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)


dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [6]:
class TextClassificationModel(nn.Module):
    
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [7]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} |"
                "| accuracy {:8.3f}".format(
                    epoch, idx, total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [20]:
emsize = 64
EPOCHS = 2  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
model = model.to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    train(dataloader)
    

In [19]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text.lower()))
        print(text)
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

In [26]:
ex_text_str = "in another news aliens took over the planet while floating in space "

print("This is a %s news" % ag_news_label[predict(ex_text_str, text_pipeline)])


tensor([  42,   50,  449,    0, 1264,   68,    7, 1568,  455, 1325,   42, 1308])
This is a Business news


In [ ]:
# Pytorch 

In [4]:
# Step 1
y = 4 * x + 3
# Step 2
x = torch.autograd.Variable(torch.Tensor([1.0]),requires_grad=True)
# Step 3
y.backward()
# Step 4 
print(x.grad)

AttributeError: 'int' object has no attribute 'backward'

In [ ]:
# Step 1: Model definition
class Net(nn.Module):
    # Define model elements
    def __init__(self, n_inputs):
        super(Net, self).__init__()
        self.layer = Linear(n_inputs, 1)
        self.activation = Sigmoid()

    # Step 2: Forward propagate input
    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

# Step 3: Test model
my_nn = Net()
result = my_nn(some_data)
print (result)

In [ ]:
running_loss = 0.

for i, data in enumerate(training_batch_set):

    # 1. Every data instance is an input + label pair
    inputs, labels = data

    # 2. Zero your gradients for every batch!
    optimizer.zero_grad()

    # 3. Make predictions for this batch
    outputs = model(inputs)

    # 4. Compute the loss and its gradients
    loss = loss_fn(outputs, labels)
    loss.backward()

    # 5. Adjust learning weights
    optimizer.step()

    # 6. Gather data and for report
    running_loss += loss.item()

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad and 'fc1' in name:
        param.requires_grad = False


In [ ]:
# Step 1
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


for epoch in range(20):
    for input, target in dataset:
        optimizer.zero_grad()
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
    # Step 2
    scheduler.step()

In [ ]:
import torch
import torch.nn as nn

# define your custom loss function
class CustomLoss(nn.Module):
    def __init__(self):
        super(MyLoss, self).__init__()
    
    def forward(self, output, target):
        # compute your custom loss function
        loss = ... # compute the loss
        
        return loss

In [ ]:
# Initialize your model and loss function
model = ...
optimizer = ...
criterion = CustomLoss()

# Training loop
for epoch in range(num_epochs):

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Rest of the code to evaluate the model goes here...	

In [ ]:
class Classify(nn.Module):
    
    def __init__(self):
        super().__init__()
        input_size = 2 
        output_size = 3
        hidden_size = 16
        self.mid_layers = []
        for _ in range(5):
            self.mid_layers.append(nn.Linear(hidden_size,hidden_size))
            
        self.mid_layers    = nn.Sequential(*self.mid_layers)
        self.input_layers  = nn.Sequential(nn.Linear(input_size,hidden_size)
                                    ,nn.ReLU())
        self.output_layers = nn.Linear(hidden_size,output_size)
                                    
    
    def forward(self,x):
        z = self.input_layers(x)
        z = self.mid_layers(z)
        z = self.output_layers(z)
        return z 
    
    
    

In [ ]:
device = 'cpu'
model = Classify()
model.to(device)

In [ ]:
input_batches =  [torch.ones((100,2),device=device) for _ in range(5)]


In [ ]:
from torch.distributions import Categorical
from torch.distributions.kl import kl_divergence

In [ ]:
dist = Categorical(logits=output)

In [ ]:
losses = []
for input_data in input_batches:
    output = model(input_data)
    criterion = nn.MSELoss(reduction='mean')
    target = torch.randn(100,3)
    loss = criterion(output,target)
    losses.append(loss.detach())

In [ ]:
model.eval()